# Top-K Similarity Search - Ask A Book A Question

In this tutorial we will see a simple example of basic retrieval via Top-K Similarity search

In [1]:
# pip install langchain --upgrade
# Version: 0.0.164

# !pip install pypdf

In [1]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()

c:\Users\anand\Desktop\ananda github\Langchain-tutorial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Load your data

Next let's load up some data. I've put a few 'loaders' on there which will load data from different locations. Feel free to use the one that suits you. The default one queries one of Paul Graham's essays for a simple example. This process will only stage the loader, not actually load it.

Then let's go ahead and actually load the data.

In [ ]:
loader = PyPDFLoader("computer_graphics_notes.json")

In [30]:
data = loader.load()

Then let's actually check out what's been loaded

In [31]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

You have 18 document(s) in your data
There are 1872 characters in your sample document
Here is a sample: https://collegenote.pythonanywhere.com                                     Prepared By: Jayanta Poudel 
 
1 Computer Graphics (Reference Note)                                                          


### Chunk your data up into smaller documents

While we could pass the entire essay to a model w/ long context, we want to be picky about which information we share with our model. The better signal to noise ratio we have the more likely we are to get the right answer.

The first thing we'll do is chunk up our document into smaller pieces. The goal will be to take only a few of those smaller pieces and pass them to the LLM.

In [130]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
texts = text_splitter.split_documents(data)

In [131]:
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 46 documents


### Create embeddings of your documents to get ready for semantic search

Next up we need to prepare for similarity searches. The way we do this is through embedding our documents (getting a vector per document).

This will help us compare documents later on.

In [132]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()


Check to see if there is an environment variable with you API keys, if not, use what you put below

### Option #1: Chroma (for local)

I like Chroma becauase it's local and easy to set up without an account.

First we'll pass our texts to Chroma via `.from_documents`, this will 1) embed the documents and get a vector, then 2) add them to the vectorstore for retrieval later.

In [133]:
# load it into Chroma
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(texts, embedding)

Let's test it out. I want to see which documents are most closely related to a query.



In [134]:
query = "What peter thiel thinks about startups?"
docs = vectorstore.similarity_search(query)

Then we can check them out. In theory, the texts which are deemed most similar should hold the answer to our question.
But keep in mind that our query just happens to be a question, it could be a random statement or sentence and it would still work.

In [135]:
# Here's an example of the first document that was returned
for doc in docs:
    print (f"{doc.page_content}\n")

command in the display list once during each refresh cycle. 
- Graphics are drawn on a vector display system by directing the electron beam along 
component lines of the picture. 
Advantages: 
- Can produce output with high resolutions. 
- Better for animation than raster system since only end point information is needed. 
 
Disadvantages: 
- Cannot fill area with pattern and manipulate bits. 
- Refreshing image depends upon its complexity.

- Graphics are drawn on a vector display system by directing the electron beam along 
component lines of the picture. 
Advantages: 
- Can produce output with high resolutions. 
- Better for animation than raster system since only end point information is needed. 
 
Disadvantages: 
- Cannot fill area with pattern and manipulate bits. 
- Refreshing image depends upon its complexity.

- Graphics are drawn on a vector display system by directing the electron beam along 
component lines of the picture. 
Advantages: 
- Can produce output with high resolu

### Option #2: Pinecone (for cloud)
If you want to use pinecone, run the code below, if not then skip over to Chroma below it. You must go to [Pinecone.io](https://www.pinecone.io/) and set up an account

In [12]:
# PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', 'YourAPIKey')
# PINECONE_API_ENV = os.getenv('PINECONE_API_ENV', 'us-east1-gcp') # You may need to switch with your env

# # initialize pinecone
# pinecone.init(
#     api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
# )
# index_name = "langchaintest" # put in the name of your pinecone index here

# docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

### Query those docs to get your answer back

Great, those are just the docs which should hold our answer. Now we can pass those to a LangChain chain to query the LLM.

We could do this manually, but a chain is a convenient helper for us.

In [70]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
    temperature=0.7)

In [136]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

template = """
You are a highly intelligent and helpful tutor.
You have access to a digital book provided as context below.

Your goal:
1. Use the information from the book as the primary reference.
2. If a direct answer is not in the book, use logical reasoning or general knowledge
   to solve or explain — but always keep it consistent with the book’s ideas.
3. If the question involves numericals, formulas, or examples, solve them clearly step-by-step.
4. If something is not mentioned in the book, explicitly state that and then provide a best logical answer.

------------------
Book Reference:
{context}
------------------

Question:
{question}

Answer (referencing the book + reasoning if needed):
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)


In [137]:
query = "Software standards ?"
docs = vectorstore.similarity_search(query,k=5)
docs

[Document(metadata={'page_label': '13', 'page': 12, 'producer': '3.0.0.M8 (5.0.0.M4)', 'creator': 'PDFium', 'creationdate': 'D:20200630190556', 'source': 'CG-1.pdf', 'moddate': '2023-02-08T15:05:22+01:00', 'total_pages': 18}, page_content='Primary goal of standardized graphics software is portability. When packages are designed \nwith standard graphics functions, software can he moved easily from one hardware system to \nanother and used in different implementations and applications. International and national \nstandards planning organizations in many countries have cooperated in an effort to develop a \ngenerally accepted standard for computer graphics. After considerable effort, this work led to \nfollowing standards:'),
 Document(metadata={'page': 12, 'total_pages': 18, 'page_label': '13', 'producer': '3.0.0.M8 (5.0.0.M4)', 'creationdate': 'D:20200630190556', 'creator': 'PDFium', 'source': 'CG-1.pdf', 'moddate': '2023-02-08T15:05:22+01:00'}, page_content="views, and applying transf

Awesome! We just went and queried an external data source!

In [138]:
chain.run(input_documents=docs, question=query)

'The primary goal of standardized graphics software is **portability**.\n\nWhen graphics packages are designed with standard graphics functions, the software can be:\n*   Moved easily from one hardware system to another.\n*   Used in different implementations and applications.\n\nInternational and national standards planning organizations in many countries have collaborated to develop generally accepted standards for computer graphics. After significant effort, this work led to specific standards (though the specific names of these standards are not fully visible in the provided excerpt beyond "GL(Graphics Library)" as an example of a general-purpose package, and the text cuts off before listing other "following standards").'